### Script para obtener indicadores de SISPLEG

In [5]:
import pandas as pd
import numpy as np
import math
import unidecode
import datetime as dt

pd.options.display.float_format = '{:,.2f}'.format

today_date = dt.datetime.now()

""" ARCHIVOS NECESITADOS """
path = 'C:\\Users\\dcabrera\\Documents\\DGCP\\DATOS'

month_string = {1:'marzo', 2:'junio', 3:'septiembre', 4:'diciembre'}

if today_date.month < 3: 
    date_string = (4, today_date.year-1)
else:
    date_string = (int((today_date.month-1)/3), today_date.year)
    
file = '\\2018\\04.consolidado_2017_2018.xlsx'
file2 = '\\{1}\\TR{0}\\Adjudicaciones\\0{0}.consolidado_{2}_{1}.xlsx'.format(date_string[0], date_string[1], month_string[date_string[0]])
file3 = '\\{1}\\TR{0}\\Proveedores\\proveedores_{2}_{1}.xlsx'.format(date_string[0], date_string[1], month_string[date_string[0]])

CIERRE2018 = pd.read_excel(path + file)
CIERRE2019 = pd.read_excel(path + file2, sheet_name = "Cierre")
PROVEEDORES = pd.read_excel(path + file3)

In [6]:
CIERRE2018 = CIERRE2018[['RPE', 'GENERO', 'APROBACION',
                         'VALOR_TOTAL', 'PROVINCIA', 'REGION', 
                         'MACROREGION', 'MODALIDAD', 'CLASIFICACION_EMPRESARIAL',
                         'CLASIFICACION_EMPRESARIAL_2']]

CIERRE2019 = CIERRE2019[['RPE', 'GENERO', 'APROBACION', 
                         'VALOR_TOTAL', 'PROVINCIA', 'REGION',
                         'MACROREGION', 'MODALIDAD', 'CLASIFICACION_EMPRESARIAL', 
                         'CLASIFICACION_EMPRESARIAL_2']]

PROVEEDORES = PROVEEDORES[['RPE', 'GENERO', 'MES', 'ANO',
                           'PROVINCIA', 'REGION', 'MACROREGION',
                           'CLASIFICACION_EMPRESARIAL', 'CLASIFICACION_EMPRESARIAL_2']]

ADJUDICACIONES = CIERRE2018.append([CIERRE2019])

In [7]:
# Creacion de la columna MACROREGION segun los lineamientos de los indicadores del SISPLEG
options = {'SUROESTE' : 'SUR', 'NORTE':'NORTE', 'No Especificada': 'No Especificada', 'SURESTE':'ESTE'}
PROVEEDORES['MACROREGION_SISPLEG'] = PROVEEDORES.apply(lambda x: 'METROPOLITANA' if x.REGION == 'OZAMA O METROPOLITANA'  
                                                       else options.get(x.MACROREGION, 'ERROR'), axis=1)
#PROVEEDORES = PROVEEDORES[PROVEEDORES.MACROREGION_SISPLEG != 'No Especificada']

ADJUDICACIONES['MACROREGION_SISPLEG'] = ADJUDICACIONES.apply(lambda x: 'METROPOLITANA' if x.REGION == 'OZAMA O METROPOLITANA' 
                                                             else options.get(x.MACROREGION, 'ERROR'), axis = 1)
#ADJUDICACIONES = ADJUDICACIONES[ADJUDICACIONES.MACROREGION_SISPLEG != 'No Especificada']

In [8]:
ADJUDICACIONES.loc[:, 'MES'] = ADJUDICACIONES.APROBACION.apply(lambda x: x.month)
ADJUDICACIONES.loc[:, 'ANO'] = ADJUDICACIONES.APROBACION.apply(lambda x: x.year)
ADJUDICACIONES.loc[:, 'TRIMESTRE'] = ADJUDICACIONES.MES.apply(lambda x: 1 if x in [1,2,3] else
                                                       (2 if x in [4,5,6] else
                                                        (3 if x in [7,8,9] else
                                                         (4 if x in [10,11,12] else 'ERROR'))))

PROVEEDORES = PROVEEDORES.rename(columns = {"Mes" : "MES", "Año" : "ANO"})
PROVEEDORES.loc[:, 'TRIMESTRE'] = PROVEEDORES.MES.apply(lambda x: 1 if x in [1,2,3] else 
                                                 (2 if x in [4,5,6] else 
                                                  (3 if x in [7,8,9] else 
                                                   (4 if x in [10,11,12] else 'ERROR'))))
PROVEEDORES = PROVEEDORES[PROVEEDORES.ANO.isin([2017, 2018, 2019])]

In [9]:
def DateREF(ano, trim):
    return max(ADJUDICACIONES.APROBACION[(ADJUDICACIONES.ANO == ano) & (ADJUDICACIONES.TRIMESTRE == trim)]).date().strftime('%d/%m/%Y')

In [10]:
columns_titles = {"ESTE" : ["NUMERO PROVEEDORES REGISTRADOS EN LA REGION ESTE", "PORCENTAJE DEL MONTO CONTRATADO EN LA REGION ESTE", 
                            "PORCENTAJE DEL MONTO CONTRATADO A MIPYME CERTIFICADAS EN LA REGION ESTE", "NUMERO DE PROVEEDORES ADJUDICADOS EN LA REGION ESTE"],
                  "METROPOLITANA" : ["NUMERO PROVEEDORES REGISTRADOS EN METROPOLITANA", "PORCENTAJE DEL MONTO CONTRATADO EN LA REGION METROPOLITANA", 
                            "PORCENTAJE DEL MONTO CONTRATADO A MIPYME CERTIFICADAS EN LA REGION METROPOLITANA", "NUMERO DE PROVEEDORES ADJUDICADOS EN LA REGION METROPOLITANA"],
                  "NORTE" : ["NUMERO PROVEEDORES REGISTRADOS EN LA REGION NORTE", "PORCENTAJE DEL MONTO CONTRATADO EN LA REGION NORTE", 
                            "PORCENTAJE DEL MONTO CONTRATADO A MIPYME CERTIFICADAS EN LA REGION NORTE", "NUMERO DE PROVEEDORES ADJUDICADOS EN LA REGION NORTE"],
                  "SUR" : ["NUMERO PROVEEDORES REGISTRADOS EN LA REGION SUR", "PORCENTAJE DEL MONTO CONTRATADO EN LA REGION SUR", 
                            "PORCENTAJE DEL MONTO CONTRATADO A MIPYME CERTIFICADAS EN LA REGION SUR", "NUMERO DE PROVEEDORES ADJUDICADOS EN LA REGION SUR"]}

In [11]:
DF = list()

for year in PROVEEDORES.ANO.drop_duplicates().values:
    for quarter in PROVEEDORES[PROVEEDORES.ANO == year].TRIMESTRE.drop_duplicates().values:
        PROVEEDORES_REGISTRADOS = pd.pivot_table(PROVEEDORES[(PROVEEDORES.ANO == year) & (PROVEEDORES.TRIMESTRE == quarter)], 
                                                 values = 'RPE', index = ['MACROREGION_SISPLEG'], aggfunc = 'count').rename(columns = {"RPE": DateREF(year, quarter)})
        
        PROVEEDORES_REGISTRADOS = PROVEEDORES_REGISTRADOS.T.rename(columns = {"ESTE" : columns_titles["ESTE"][0],
                                                                             "METROPOLITANA" : columns_titles["METROPOLITANA"][0],
                                                                             "NORTE" : columns_titles["NORTE"][0],
                                                                             "SUR" : columns_titles["SUR"][0]})
        PROVEEDORES_REGISTRADOS.drop(['No Especificada'], axis=1, inplace=True)
        
        PROVEEDORES_ADJUDICADOS = ADJUDICACIONES[(ADJUDICACIONES.ANO == year) & (ADJUDICACIONES.TRIMESTRE == quarter)].drop_duplicates(subset = 'RPE', keep = 'first')\
                                    .pivot_table(values = 'RPE', index = 'MACROREGION_SISPLEG', aggfunc='count').rename(columns = {"RPE": DateREF(year, quarter)})

        PROVEEDORES_ADJUDICADOS = PROVEEDORES_ADJUDICADOS.T.rename(columns = {"ESTE" : columns_titles["ESTE"][3],
                                                                             "METROPOLITANA" : columns_titles["METROPOLITANA"][3],
                                                                             "NORTE" : columns_titles["NORTE"][3],
                                                                             "SUR" : columns_titles["SUR"][3]})
        PROVEEDORES_ADJUDICADOS.drop(['No Especificada'], axis=1, inplace=True)
        
        MONTO_TOTAL = ADJUDICACIONES[(ADJUDICACIONES.ANO == year) & (ADJUDICACIONES.TRIMESTRE == quarter)].VALOR_TOTAL.sum()
        MONTO_CONTRATADO = ADJUDICACIONES[(ADJUDICACIONES.ANO == year) & (ADJUDICACIONES.TRIMESTRE == quarter)].pivot_table(values = 'VALOR_TOTAL',
                                                                                                         index=['MACROREGION_SISPLEG'], 
                                                                                                         aggfunc =  lambda x : (x.sum()/MONTO_TOTAL))\
                                                                                                        .rename(columns = {"VALOR_TOTAL": DateREF(year, quarter)})
        
        MONTO_CONTRATADO = MONTO_CONTRATADO.T.rename(columns = {"ESTE" : columns_titles["ESTE"][1],
                                                                "METROPOLITANA" : columns_titles["METROPOLITANA"][1],
                                                                "NORTE" : columns_titles["NORTE"][1],
                                                                "SUR" : columns_titles["SUR"][1]})
        
        MONTO_CONTRATADO.drop(['No Especificada'], axis=1, inplace=True)
        MONTO_CONTRATADO['MONTO TOTAL CONTRATADO'] = ADJUDICACIONES[(ADJUDICACIONES.ANO == year) & (ADJUDICACIONES.TRIMESTRE == quarter)].groupby(['ANO'])['VALOR_TOTAL'].sum().values[0]
        
        MONTO_MIPYMES = ADJUDICACIONES[(ADJUDICACIONES.CLASIFICACION_EMPRESARIAL_2 == 'MIPYMES Certificadas por el MIC') & ((ADJUDICACIONES.ANO == year) & (ADJUDICACIONES.TRIMESTRE == quarter))]\
                        .pivot_table(values = 'VALOR_TOTAL', index=['MACROREGION_SISPLEG'], 
                                     aggfunc = lambda x : (x.sum()/MONTO_TOTAL)).rename(columns = {"VALOR_TOTAL": DateREF(year, quarter)})

        
        MONTO_MIPYMES = MONTO_MIPYMES.T.rename(columns = {"ESTE" : columns_titles["ESTE"][2],
                                                          "METROPOLITANA" : columns_titles["METROPOLITANA"][2],
                                                          "NORTE" : columns_titles["NORTE"][2],
                                                          "SUR" : columns_titles["SUR"][2]})
        if 'No Especificada' in MONTO_MIPYMES.columns:
            MONTO_MIPYMES.drop(['No Especificada'], axis=1, inplace=True)
        
        PROVEEDORES_REGISTRADOS["NUMERO DE PROVEEDORES REGISTRADOS QUE CORRESPONDEN A MIPYMES"] = pd.pivot_table(PROVEEDORES[(PROVEEDORES.ANO == year) & (PROVEEDORES.TRIMESTRE == quarter)], 
                                                                                                                   values = 'RPE', index = ['CLASIFICACION_EMPRESARIAL_2'], 
                                                                                                                   aggfunc = 'count').loc['MIPYMES Certificadas por el MIC'].values[0]
        
        PROVEEDORES_REGISTRADOS["NUMERO DE PROVEEDORES REGISTRADOS TOTAL"] = PROVEEDORES[(PROVEEDORES.ANO == year) & (PROVEEDORES.TRIMESTRE == quarter)].groupby(['TRIMESTRE'])['RPE'].count().values[0]

        DF.append(pd.concat([PROVEEDORES_REGISTRADOS, PROVEEDORES_ADJUDICADOS, MONTO_CONTRATADO, MONTO_MIPYMES], axis = 1, sort=False))

INDICADORES_SISPLEG = pd.concat(DF, sort=False)

In [13]:
INDICADORES_SISPLEG_2 = INDICADORES_SISPLEG.unstack().reset_index().rename(columns = {'MACROREGION_SISPLEG': 'INDICADORES_SISPLEG', 'level_1':'Fecha', 0:'ind'})

In [17]:
writer = pd.ExcelWriter('indicadores_sispleg.xlsx')
INDICADORES_SISPLEG.reset_index().to_excel(writer, index = False, sheet_name='Indicadores')
INDICADORES_SISPLEG_2.reset_index().to_excel(writer, index = False, sheet_name='Page2')
writer.save()